In [ ]:
!pip install -q condacolab

In [1]:
%load_ext autoreload
%autoreload 2
import os
import time
import jax

import numpy as np
import pandas as pd
import sys
import yaml
jax.config.update("jax_enable_x64", True)
import pickle

In [ ]:
# Install dependencies
!pip install linearmodels

In [ ]:
import condacolab
condacolab.install()
condacolab.check()

In [ ]:
!conda install estimagic -c conda-forge

In [ ]:
import estimagic as em

In [2]:
# Mount drive. Make sure everything(including submodules) are there
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/colab_notebooks/policy_unc/analysis

/content/drive/MyDrive/colab_notebooks/policy_unc/analysis


In [ ]:
analysis_path = os.getcwd() + "/"
sys.path.insert(0, analysis_path + "submodules/dcegm/src/")
sys.path.insert(0, analysis_path + "src/")
from set_paths import create_path_dict
path_dict = create_path_dict(analysis_path)

In [ ]:
from estimation.estimate_setup import estimate_model
estimate_model(path_dict, load_model=True)